In [1]:
import pandas as pd
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model

In [2]:
base = pd.read_csv('games.csv')
base.head(5)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Global_Sales', axis = 1)
base = base.drop('Developer', axis = 1)

In [7]:
base = base.dropna(axis = 0) # remove all rows containing NaN values

In [9]:
base = base.loc[base['NA_Sales'] > 1]
base = base.loc[base['EU_Sales'] > 1]

In [10]:
base['Name'].value_counts()

Grand Theft Auto V                 4
Call of Duty: Ghosts               3
The Elder Scrolls V: Skyrim        3
Battlefield 4                      3
Assassin's Creed IV: Black Flag    3
                                  ..
Kinect Adventures!                 1
Final Fantasy VIII                 1
Max Payne                          1
Wii Play                           1
Spider-Man: The Movie              1
Name: Name, Length: 223, dtype: int64

In [13]:
nome_jogos = base.Name
nome_jogos.head(10)

0                    Wii Sports
2                Mario Kart Wii
3             Wii Sports Resort
6         New Super Mario Bros.
7                      Wii Play
8     New Super Mario Bros. Wii
11                Mario Kart DS
13                      Wii Fit
14           Kinect Adventures!
15                 Wii Fit Plus
Name: Name, dtype: object

In [14]:
base = base.drop('Name', axis = 1)
base.head(4)

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
2,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
6,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,89.0,65.0,8.5,431.0,E


In [15]:
previsores = base.iloc[:, [0,1,2,3,7,8,9,10,11]].values
venda_na = base.iloc[:, 4].values
venda_eu = base.iloc[:, 5].values
venda_jp = base.iloc[:, 6].values

In [16]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
 
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,2,3,8])],remainder='passthrough')
previsores = onehotencorder.fit_transform(previsores).toarray()

In [17]:
previsores.shape

(258, 61)

In [18]:
camada_entrada = Input(shape=(61,))

camada_oculta1 = Dense(units = 32, # (61+3)/2 = 32
                       activation = 'sigmoid')(camada_entrada)

camada_oculta2 = Dense(units = 32, activation = 'sigmoid')(camada_oculta1)

camada_saida1 = Dense(units = 1, activation = 'linear')(camada_oculta2)

camada_saida2 = Dense(units = 1, activation = 'linear')(camada_oculta2)

camada_saida3 = Dense(units = 1, activation = 'linear')(camada_oculta2)

regressor = Model(inputs = camada_entrada,
                  outputs = [camada_saida1, camada_saida2, camada_saida3])
regressor.compile(optimizer = 'adam',
                  loss = 'mse')

regressor.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 61)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           1984        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 32)           1056        dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1)            33          dense_1[0][0]                    
_______________________________________________________________________________________

In [ ]:
regressor.fit(previsores, [venda_na, venda_eu, venda_jp],
              epochs = 5000, batch_size = 100)

Epoch 1/5000
3/3 [==============================] - 0s 2ms/step - loss: 28.3389 - dense_2_loss: 15.3170 - dense_3_loss: 11.8181 - dense_4_loss: 1.2039
Epoch 2/5000
3/3 [==============================] - 0s 2ms/step - loss: 27.3965 - dense_2_loss: 14.9923 - dense_3_loss: 11.2122 - dense_4_loss: 1.1921
Epoch 3/5000
3/3 [==============================] - 0s 2ms/step - loss: 26.5117 - dense_2_loss: 14.6860 - dense_3_loss: 10.6404 - dense_4_loss: 1.1853
Epoch 4/5000
3/3 [==============================] - 0s 2ms/step - loss: 25.6858 - dense_2_loss: 14.3972 - dense_3_loss: 10.1040 - dense_4_loss: 1.1845
Epoch 5/5000
3/3 [==============================] - 0s 2ms/step - loss: 24.9324 - dense_2_loss: 14.1418 - dense_3_loss: 9.6076 - dense_4_loss: 1.1831
Epoch 6/5000
3/3 [==============================] - 0s 2ms/step - loss: 24.2161 - dense_2_loss: 13.8820 - dense_3_loss: 9.1486 - dense_4_loss: 1.1855
Epoch 7/5000
3/3 [==============================] - 0s 2ms/step - loss: 23.5370 - dense_2_loss: 

In [20]:
previsao_na, previsao_eu, previsao_jp = regressor.predict(previsores)

In [22]:
previsao_na[0:10]

array([[27.127491],
       [13.620224],
       [12.935449],
       [ 9.491318],
       [ 4.793316],
       [ 8.491419],
       [ 7.336504],
       [12.74884 ],
       [ 4.409836],
       [ 9.978926]], dtype=float32)

In [25]:
venda_na[0:10]

array([41.36, 15.68, 15.61, 11.28, 13.96, 14.44,  9.71,  8.92, 15.  ,
        9.01])